In [1]:
from bdc_downloader import *

In [2]:
mt_data = get_municipality_data("MT")
campo_verde = mt_data[mt_data["name_muni"] == "Campo Verde"]

In [3]:
campo_verde.to_file("./dados/campo_verde.geojson", driver="GeoJSON")
campo_verde

,code_muni,name_muni,code_state,abbrev_state,name_state,code_region,name_region,geometry
22,5102678.0,Campo Verde,51.0,MT,Mato Grosso,5.0,Centro Oeste,"MULTIPOLYGON (((-54.69703 -14.95348, -54.69683..."


In [4]:
bbox = campo_verde.total_bounds.tolist()

In [5]:
client = get_stac_client()
collections_available = get_available_collections(client)

In [6]:
s2_collections = [c for c in collections_available if "s2" in c.lower()]
# for collection in s2_collections:
#     print(collection)
#     print(get_collection_assets_metadata(collection, client).keys())
#     available_dates = get_collection_available_dates(collection, client)
#     print(available_dates[0].date(), "-", available_dates[-1].date())

#     print()

In [7]:
cerrado_4m = "mosaic-s2-cerrado-4m-1"

cerrado_2m = "mosaic-s2-cerrado-2m-1"

In [8]:
get_collection_assets_metadata(cerrado_4m, client)

{'visual': {'title': 'False Color Composition of the mosaic. Contains Sentinel-2 Bands B04, B08 and B03.',
  'description': None,
  'media_type': 'image/tiff; application=geotiff; profile=cloud-optimized',
  'roles': ['data']},
 'thumbnail': {'title': 'Thumbnail',
  'description': None,
  'media_type': 'image/png',
  'roles': ['thumbnail']}}

In [9]:
get_collection_assets_metadata(cerrado_2m, client)

{'visual': {'title': 'False Color Composition of the mosaic. Contains Sentinel-2 Bands B04, B08 and B03.',
  'description': None,
  'media_type': 'image/tiff; application=geotiff; profile=cloud-optimized',
  'roles': ['data']},
 'thumbnail': {'title': 'Thumbnail',
  'description': None,
  'media_type': 'image/png',
  'roles': ['thumbnail']}}

In [10]:
date_range = "2017-07-12/2025-07-13"
assets = "visual"

In [11]:
items = search_stac_items(
    cerrado_4m,
    bbox=bbox,
    datetime=date_range,
    cloud_cover_lt=10000,
    limit=50,
    client=client,
)
print(f"Found {len(items)} items in the collection {cerrado_4m}.")

Found 6 items in the collection mosaic-s2-cerrado-4m-1.


In [12]:
from threading import Thread

threads = []
for item in items:
    thread = Thread(
        target=download_item_assets,
        args=(
            item,
            "./dados",
            assets,
        ),
    )
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

Download do asset 'visual' concluído.
Download do asset 'visual' concluído.
Download do asset 'visual' concluído.
Download do asset 'visual' concluído.
Download do asset 'visual' concluído.
Download do asset 'visual' concluído.


In [13]:
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
import glob
import os

mosaic_pattern = os.path.join("./dados", "MOSAIC-S2-CERRADO-4M*", "*VISUAL.tif")
mosaic_files = glob.glob(mosaic_pattern)
print(f"Encontrados {len(mosaic_files)} mosaicos.")

datasets = [rasterio.open(fp) for fp in mosaic_files]
merged_array, merged_transform = merge(datasets)
profile = datasets[0].profile.copy()
profile.update({
    'height': merged_array.shape[1],
    'width': merged_array.shape[2],
    'transform': merged_transform
})
for ds in datasets:
    ds.close()

if campo_verde.crs != profile['crs']:
    aoi = campo_verde.to_crs(profile['crs'])
else:
    aoi = campo_verde

with rasterio.io.MemoryFile() as memfile:
    with memfile.open(**profile) as merged_ds:
        merged_ds.write(merged_array)
        clipped_array, clipped_transform = mask(merged_ds, aoi.geometry, crop=True)
        out_profile = merged_ds.profile.copy()

out_profile.update({
    'height': clipped_array.shape[1],
    'width': clipped_array.shape[2],
    'transform': clipped_transform
})

output_path = "./dados/campo_verde_merged_clip.tif"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with rasterio.open(output_path, 'w', **out_profile) as dst:
    dst.write(clipped_array)

print(f"Mosaico concatenado e recortado salvo em: {output_path}")

Encontrados 6 mosaicos.
Mosaico concatenado e recortado salvo em: ./dados/campo_verde_merged_clip.tif


In [23]:
from rasterio.windows import Window
import numpy as np

def criar_patches(imagem_tif_path, saida_dir, tamanho_patch=1024, sobreposicao=512):
    if not os.path.exists(saida_dir):
        os.makedirs(saida_dir)
        print(f"Diretório criado: {saida_dir}")

    print(f"Processando a imagem: {imagem_tif_path}")
    with rasterio.open(imagem_tif_path) as src:
        altura, largura = src.height, src.width
        transform_original = src.transform
        crs_original = src.crs
        profile = src.profile.copy()

        print(f"Dimensões da imagem: {largura} x {altura}")
        print(f"CRS: {crs_original}")

        passo = tamanho_patch - sobreposicao
        nome_base = os.path.splitext(os.path.basename(imagem_tif_path))[0]
        patches_criados = 0

        for i in range(0, altura, passo):
            for j in range(0, largura, passo):
                col_off, row_off = j, i
                col_size = min(tamanho_patch, largura - j)
                row_size = min(tamanho_patch, altura - i)

                if col_size < tamanho_patch // 2 or row_size < tamanho_patch // 2:
                    continue

                window = Window(col_off, row_off, col_size, row_size)
                patch_data = src.read(window=window)

                if np.all(patch_data == 0):
                    continue

                patch_transform = rasterio.windows.transform(window, transform_original)
                patch_profile = profile.copy()
                patch_profile.update({
                    'height': row_size,
                    'width': col_size,
                    'transform': patch_transform,
                })

                nome_patch = f"{nome_base}_bloco_{row_off}_{col_off}.tif"
                caminho_saida = os.path.join(saida_dir, nome_patch)
                with rasterio.open(caminho_saida, 'w', **patch_profile) as dst:
                    dst.write(patch_data)

                patches_criados += 1

        print(f"Total de {patches_criados} patches salvos em {saida_dir}")

criar_patches(output_path, "./dados/patches_campo_verde", tamanho_patch=1028, sobreposicao=64)

Diretório criado: ./dados/patches_campo_verde
Processando a imagem: ./dados/campo_verde_merged_clip.tif
Dimensões da imagem: 12476 x 11651
CRS: PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown_based_on_GRS80_ellipsoid",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",-12],PARAMETER["longitude_of_center",-54],PARAMETER["standard_parallel_1",-2],PARAMETER["standard_parallel_2",-22],PARAMETER["false_easting",5000000],PARAMETER["false_northing",10000000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Total de 84 patches salvos em ./dados/patches_campo_verde


In [15]:
from urllib.request import urlretrieve

In [16]:
link_map_biomas = "https://storage.googleapis.com/mapbiomas-public/initiatives/brasil/lulc_10m/collection_2/integration/mapbiomas_10m_collection2_integration_v1-classification_2022.tif"
path = (
    "./dados/mapbiomas_10m_collection2_integration_v1-classification_2022.tif"
)

In [17]:
urlretrieve(link_map_biomas, path)

('./dados/mapbiomas_10m_collection2_integration_v1-classification_2022.tif',
 <http.client.HTTPMessage at 0x7780a2839710>)

In [18]:
with rasterio.open(path) as src:
    if aoi.crs != src.crs:
        print("CRS são diferentes. Reprojetando o shapefile...")
        aoi_reprojetado = aoi.to_crs(src.crs)
    else:
        aoi_reprojetado = aoi
    
    clipped_mb, clipped_transform = mask(
        src, aoi_reprojetado.geometry, crop=True
    )

    mb_profile = src.profile.copy()

mb_profile.update({
    "height": clipped_mb.shape[1],
    "width": clipped_mb.shape[2],
    "transform": clipped_transform
})

output_mb = "./dados/mapbiomas_campo_verde.tif"
os.makedirs(os.path.dirname(output_mb), exist_ok=True)
with rasterio.open(output_mb, "w", **mb_profile) as dst:
    dst.write(clipped_mb)

os.remove(path)

print(f"MapBiomas recortado salvo em: {output_mb}")

CRS são diferentes. Reprojetando o shapefile...
MapBiomas recortado salvo em: ./dados/mapbiomas_campo_verde.tif
